In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glove-vec/glove.6B.100d.word2vec
/kaggle/input/commonlitreadabilityprize/sample_submission.csv
/kaggle/input/commonlitreadabilityprize/train.csv
/kaggle/input/commonlitreadabilityprize/test.csv
/kaggle/input/roberta-base/config.json
/kaggle/input/roberta-base/merges.txt
/kaggle/input/roberta-base/vocab.json
/kaggle/input/roberta-base/pytorch_model.bin


In [2]:
import pandas
import numpy


df = pandas.read_csv("../input/commonlitreadabilityprize/train.csv")
num_records = len(df)

ids = numpy.arange(num_records)
ids = numpy.random.permutation(ids)

train_size = 0.8
partition = int(num_records * train_size)

train_ids, valid_ids = ids[:partition], ids[partition:]

df.loc[train_ids].to_csv("./processed_train.csv", index=False)
df.loc[valid_ids].to_csv("./processed_valid.csv", index=False)


In [3]:
from typing import Any, Dict, Iterable, MutableMapping, Optional
from urllib.parse import urlparse

from allennlp.data import DatasetReader
from allennlp.data import Tokenizer
from allennlp.data.fields.field import Field
from allennlp.data.fields import ArrayField, TextField
from allennlp.data.instance import Instance
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.token_indexers.token_indexer import TokenIndexer
import pandas
import numpy
from overrides import overrides


@DatasetReader.register("commonlit_reader")
class CommonlitDatasetReader(DatasetReader):
    def __init__(
        self,
        tokenizer: Tokenizer,
        token_indexers: Optional[Dict[str, TokenIndexer]] = None,
    ) -> None:

        super().__init__()

        self.tokenizer = tokenizer
        self.token_indexers: Dict[str, TokenIndexer] = token_indexers or {
            "tokens": SingleIdTokenIndexer(),
        }

    def _read(self, file_path: str) -> Iterable[Instance]:
        instances = []

        dataframe = pandas.read_csv(file_path)
        dataframe["hostname"] = dataframe \
            .url_legal \
            .apply(lambda url: urlparse(url).hostname if isinstance(url, str) else "EMPTY_HOSTNAME")

        for _, row in dataframe.iterrows():
            excerpt = row.excerpt
            target = row.target if hasattr(row, "target") else None
            instances.append(self.text_to_instance(excerpt, target))

        return instances

    @overrides
    def text_to_instance(self, excerpt: str, target: Optional[float] = None) -> Instance:
        tokens = self.tokenizer.tokenize(excerpt)
        fields: MutableMapping[str, Field[Any]] = {
            "excerpt": TextField(tokens),
        }
        if target is not None:
            fields["target"] = ArrayField(numpy.asarray(target, dtype=numpy.float32))
        return Instance(fields=fields)

    def apply_token_indexers(self, instance: Instance) -> None:
        assert isinstance(instance.fields["excerpt"], TextField)
        instance.fields["excerpt"].token_indexers = self.token_indexers

        
from typing import Dict, Optional
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules import TextFieldEmbedder
from allennlp.modules import Seq2VecEncoder
from allennlp.nn.util import get_text_field_mask
from allennlp.data.fields.text_field import TextFieldTensors
from overrides.overrides import overrides
from torch import FloatTensor
from torch.functional import Tensor
from torch.nn.functional import mse_loss
from torch import sqrt
from torch.nn import Linear


EPS = 1e-8


@Model.register("baseline")
class BaselineRegressor(Model):

    def __init__(
        self,
        vocab: Vocabulary,
        excerpt_embedder: TextFieldEmbedder,
        excerpt_encoder: Seq2VecEncoder,
    ) -> None:

        super().__init__(vocab)

        self.vocab = vocab
        self.excerpt_embedder = excerpt_embedder
        self.excerpt_encoder = excerpt_encoder

        self.dense = Linear(
            in_features=self.excerpt_encoder.get_output_dim(),
            out_features=1,
        )

    @overrides
    def forward(
        self,
        excerpt: TextFieldTensors,
        target: Optional[FloatTensor] = None,
    ) -> Dict[str, Tensor]:

        mask = get_text_field_mask(excerpt)
        excerpt_emb = self.excerpt_embedder(excerpt)
        logit = self.dense(self.excerpt_encoder(excerpt_emb, mask=mask))

        output_dict = {"logit": logit}
        if target is not None:
            output_dict["loss"] = sqrt(mse_loss(logit.view(-1), target) + EPS)

        return output_dict

    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        return {}

    
from allennlp.common.util import JsonDict
from allennlp.data.instance import Instance
from allennlp.predictors import Predictor


@Predictor.register("regressor_predictor")
class RegressorPredictor(Predictor):
    def _json_to_instance(self, json_dict: JsonDict) -> Instance:
        return self._dataset_reader.text_to_instance(**json_dict)  # type: ignore

In [4]:
!ls ../input

commonlitreadabilityprize  glove-vec  roberta-base


In [5]:
jsonnet_text = """\
{
    dataset_reader: {
        type: "commonlit_reader",
        tokenizer: {
            type: "pretrained_transformer",
            model_name: "../input/roberta-base",
        },
        token_indexers: {
            tokens: {
                type: "pretrained_transformer",
                model_name: "../input/roberta-base",
            },
        },
    },
    train_data_path: "./processed_train.csv",
    validation_data_path: "./processed_valid.csv",
    model: {
        type: "baseline",
        excerpt_embedder: {
            type: "basic",
            token_embedders: {
                tokens: {
                    type: "pretrained_transformer",
                    model_name: "../input/roberta-base",
                },
            },
        },
        excerpt_encoder: {
            type: "bert_pooler",
            pretrained_model: "../input/roberta-base",
        }
    },
    trainer: {
        num_epochs: 30,
        learning_rate_scheduler: {
            type: "slanted_triangular",
            num_epochs: 10,
            num_steps_per_epoch: 3088,
            cut_frac: 0.06
        },
        optimizer: {
            type: "huggingface_adamw",
            lr: 2e-5,
            weight_decay: 0.1,
        },
        validation_metric: "-loss"
    },
    data_loader: {
        batch_size: 16,
        shuffle: true
    }
}

"""

f = open("baseline.jsonnet", "w")
f.write(jsonnet_text)
f.close()

In [ ]:
import allennlp.commands

allennlp.commands.train.train_model_from_file(
    parameter_filename="./baseline.jsonnet",
    serialization_dir="./serialization/4",
)

2021-07-26 13:39:30,443 - INFO - allennlp.common.params - random_seed = 13370
2021-07-26 13:39:30,444 - INFO - allennlp.common.params - numpy_seed = 1337
2021-07-26 13:39:30,445 - INFO - allennlp.common.params - pytorch_seed = 133
2021-07-26 13:39:30,495 - INFO - allennlp.common.checks - Pytorch version: 1.7.0
2021-07-26 13:39:30,497 - INFO - allennlp.common.params - type = default
2021-07-26 13:39:30,499 - INFO - allennlp.common.params - dataset_reader.type = commonlit_reader
2021-07-26 13:39:30,501 - INFO - allennlp.common.params - dataset_reader.tokenizer.type = pretrained_transformer
2021-07-26 13:39:30,502 - INFO - allennlp.common.params - dataset_reader.tokenizer.model_name = ../input/roberta-base
2021-07-26 13:39:30,504 - INFO - allennlp.common.params - dataset_reader.tokenizer.add_special_tokens = True
2021-07-26 13:39:30,505 - INFO - allennlp.common.params - dataset_reader.tokenizer.max_length = None
2021-07-26 13:39:30,506 - INFO - allennlp.common.params - dataset_reader.toke

loading instances: 0it [00:00, ?it/s]

2021-07-26 13:39:36,497 - INFO - allennlp.common.params - data_loader.type = multiprocess
2021-07-26 13:39:36,499 - INFO - allennlp.common.params - data_loader.batch_size = 16
2021-07-26 13:39:36,501 - INFO - allennlp.common.params - data_loader.drop_last = False
2021-07-26 13:39:36,502 - INFO - allennlp.common.params - data_loader.shuffle = True
2021-07-26 13:39:36,503 - INFO - allennlp.common.params - data_loader.batch_sampler = None
2021-07-26 13:39:36,504 - INFO - allennlp.common.params - data_loader.batches_per_epoch = None
2021-07-26 13:39:36,505 - INFO - allennlp.common.params - data_loader.num_workers = 0
2021-07-26 13:39:36,506 - INFO - allennlp.common.params - data_loader.max_instances_in_memory = None
2021-07-26 13:39:36,507 - INFO - allennlp.common.params - data_loader.start_method = fork
2021-07-26 13:39:36,508 - INFO - allennlp.common.params - data_loader.cuda_device = None
2021-07-26 13:39:36,509 - INFO - allennlp.common.params - data_loader.quiet = False
2021-07-26 13:3

loading instances: 0it [00:00, ?it/s]

2021-07-26 13:39:37,692 - INFO - allennlp.common.params - type = from_instances
2021-07-26 13:39:37,693 - INFO - allennlp.common.params - min_count = None
2021-07-26 13:39:37,694 - INFO - allennlp.common.params - max_vocab_size = None
2021-07-26 13:39:37,695 - INFO - allennlp.common.params - non_padded_namespaces = ('*tags', '*labels')
2021-07-26 13:39:37,696 - INFO - allennlp.common.params - pretrained_files = None
2021-07-26 13:39:37,697 - INFO - allennlp.common.params - only_include_pretrained_words = False
2021-07-26 13:39:37,697 - INFO - allennlp.common.params - tokens_to_add = None
2021-07-26 13:39:37,698 - INFO - allennlp.common.params - min_pretrained_embeddings = None
2021-07-26 13:39:37,699 - INFO - allennlp.common.params - padding_token = @@PADDING@@
2021-07-26 13:39:37,705 - INFO - allennlp.common.params - oov_token = @@UNKNOWN@@
2021-07-26 13:39:37,711 - INFO - allennlp.data.vocabulary - Fitting token dictionary from dataset.


...

  0%|          | 0/142 [00:00<?, ?it/s]

2021-07-26 13:39:53,922 - INFO - allennlp.training.callbacks.console_logger - Batch inputs
2021-07-26 13:39:53,923 - INFO - allennlp.training.callbacks.console_logger - batch_input/excerpt/tokens/token_ids (Shape: 16 x 248)
tensor([[    0,  2409,     5,  ...,     1,     1,     1],
        [    0,  3762,   183,  ...,     1,     1,     1],
        [    0,  4148,     5,  ...,     1,     1,     1],
        ...,
        [    0,  9962,  1141,  ...,     1,     1,     1],
        [    0, 32136,   155,  ...,     1,     1,     1],
        [    0,  3750,   204,  ...,     1,     1,     1]], device='cuda:0')
2021-07-26 13:39:53,930 - INFO - allennlp.training.callbacks.console_logger - batch_input/excerpt/tokens/mask (Shape: 16 x 248)
tensor([[ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False],
        ...,
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  

  0%|          | 0/36 [00:00<?, ?it/s]

2021-07-26 13:40:58,111 - INFO - allennlp.training.callbacks.console_logger - Batch inputs
2021-07-26 13:40:58,112 - INFO - allennlp.training.callbacks.console_logger - batch_input/excerpt/tokens/token_ids (Shape: 16 x 258)
tensor([[    0,   133,  3939,  ...,     1,     1,     1],
        [    0,  4688, 40701,  ...,     1,     1,     1],
        [    0, 13624,    52,  ...,     2,     1,     1],
        ...,
        [    0, 38056,    74,  ...,     1,     1,     1],
        [    0,   133,  1385,  ...,     1,     1,     1],
        [    0,   243,    21,  ...,     1,     1,     1]], device='cuda:0')
2021-07-26 13:40:58,118 - INFO - allennlp.training.callbacks.console_logger - batch_input/excerpt/tokens/mask (Shape: 16 x 258)
tensor([[ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ...,  True, False, False],
        ...,
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  

  0%|          | 0/142 [00:00<?, ?it/s]

In [ ]:
from allennlp.models.archival import load_archive

archive = load_archive("serialization/2/model.tar.gz")
predictor = RegressorPredictor.from_archive(archive)

In [ ]:
test_df = pandas.read_csv("/kaggle/input/commonlitreadabilityprize/test.csv")
print(test_df.head())

batch_json = test_df.excerpt.apply(lambda excerpt: {"excerpt": excerpt}).tolist()
predictor.predict_batch_json(batch_json)

In [ ]:
class BatchIterator:
        def __init__(self, data, batch_size):
                self.data = data
                self.batch_size = batch_size
                self.cur = 0
            
        def __iter__(self):
                return self
            
        def __next__(self):
                batch = self.data[self.cur:self.cur+self.batch_size]
                self.cur += self.batch_size
                if len(batch) == 0:
                    raise StopIteration
                return batch


predictions = []
batch_iterator = BatchIterator(batch_json, batch_size=32)

for batch in batch_iterator:
    predictions += predictor.predict_batch_json(batch)

In [ ]:
test_df["target"] = list(map(lambda p: p["logit"][0], predictions))
test_df[["id", "target"]].to_csv("submission.csv", index=False)
test_df